In [ ]:
!pip install -q kaggle
from google.colab import files
uploaded = files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# https://www.kaggle.com/datasets/sriharshaeedala/financial-fraud-detection-dataset/data
!kaggle datasets download -d sriharshaeedala/financial-fraud-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/sriharshaeedala/financial-fraud-detection-dataset
License(s): CC-BY-SA-4.0
100% 178M/178M [00:09<00:00, 28.6MB/s]
100% 178M/178M [00:09<00:00, 19.8MB/s]


In [ ]:
!unzip financial-fraud-detection-dataset.zip


Archive:  financial-fraud-detection-dataset.zip
  inflating: Synthetic_Financial_datasets_log.csv  


In [ ]:
import pandas as pd

df = pd.read_csv('/content/Synthetic_Financial_datasets_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
# selecting the columns of numerical type
numeric_columns = df.select_dtypes(include=['int', 'float']).columns

numeric_data = df[numeric_columns]

# pearson corrleation matrix of the numerical data
correlation = numeric_data.corr()
target_correlation = correlation['isFraud'].drop('isFraud')
threshold= 0.02
features=target_correlation[abs(target_correlation) >= threshold].index
features=features[:2]
copy_df=df[features].join(df['type'])
copy_df=copy_df.join(df['isFraud'])
df=copy_df
df.head()

,step,amount,type,isFraud
0,1,9839.64,PAYMENT,0
1,1,1864.28,PAYMENT,0
2,1,181.00,TRANSFER,1
3,1,181.00,CASH_OUT,1
4,1,11668.14,PAYMENT,0


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])

In [ ]:
scaler = MinMaxScaler()
df['amount'] = scaler.fit_transform(df[['amount']])

In [ ]:
class_0_df = df[df['isFraud'] == 0]
print(class_0_df)

         step    amount  type  isFraud
0           1  0.000106     3        0
1           1  0.000020     3        0
4           1  0.000126     3        0
5           1  0.000085     3        0
6           1  0.000077     3        0
...       ...       ...   ...      ...
6362319   718  0.000093     3        0
6362320   718  0.001722     1        0
6362321   718  0.002015     1        0
6362322   718  0.000888     4        0
6362323   718  0.000020     2        0

[6354407 rows x 4 columns]


In [ ]:
X = df.drop('isFraud', axis=1)
y = df['isFraud']

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the generator network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_smote.values, dtype=torch.float32)
y_tensor = torch.tensor(y_smote.values, dtype=torch.float32)
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Initialize the generator and discriminator
input_dim = X_tensor.shape[1]
output_dim = X_tensor.shape[1]
generator = Generator(input_dim, output_dim)
discriminator = Discriminator(input_dim)

# Define optimizers
optimizer_g = optim.Adam(generator.parameters(), lr=0.001)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.001)

# Define loss function
criterion = nn.BCELoss()

# Training loop
epochs = 100
for epoch in range(epochs):
    for real_data, _ in dataloader:
        # Train discriminator
        optimizer_d.zero_grad()

        # Real data loss
        real_labels = torch.ones(real_data.size(0), 1)
        real_output = discriminator(real_data)
        d_loss_real = criterion(real_output, real_labels)

        # Generate fake data
        noise = torch.randn(real_data.size(0), input_dim)
        fake_data = generator(noise)
        fake_labels = torch.zeros(real_data.size(0), 1)
        fake_output = discriminator(fake_data.detach())
        d_loss_fake = criterion(fake_output, fake_labels)

        # Total discriminator loss
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_d.step()

        # Train generator
        optimizer_g.zero_grad()
        fake_output = discriminator(fake_data)
        g_loss = criterion(fake_output, real_labels)  # Aim to fool the discriminator
        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch [{epoch+1}/{epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

# Generate synthetic data
noise = torch.randn(100, input_dim)
synthetic_data = generator(noise).detach().numpy()

# Evaluate the accuracy of the discriminator on the generated data
fake_output = discriminator(torch.tensor(synthetic_data, dtype=torch.float32))
accuracy = (fake_output > 0.5).sum().item() / len(fake_output)

print(f"Discriminator Accuracy on Synthetic Data: {accuracy:.4f}")


Epoch [1/100], D Loss: 0.0016, G Loss: 6.4108
Epoch [2/100], D Loss: 0.0008, G Loss: 7.1393
Epoch [3/100], D Loss: 0.0000, G Loss: 10.5735
Epoch [4/100], D Loss: 0.0000, G Loss: 11.4365
Epoch [5/100], D Loss: 0.0000, G Loss: 11.4959
Epoch [6/100], D Loss: 0.0009, G Loss: 7.0343
Epoch [7/100], D Loss: 0.0006, G Loss: 7.4963
Epoch [8/100], D Loss: 0.0949, G Loss: 7.3211
Epoch [9/100], D Loss: 0.0007, G Loss: 7.2068
Epoch [10/100], D Loss: 0.0009, G Loss: 7.0440
